<a href="https://colab.research.google.com/github/frankssenoga/frankssenoga/blob/main/SOM%20implementation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install minisom

In [ ]:
import math
import numpy as np
from scipy.sparse.csgraph import connected_components
from minisom import MiniSom
import plotly.graph_objects as go
import matplotlib.pyplot as plt

# Function to calculate the number of unique paths considering connectivity
def tsp_unique_paths(n, distance_matrix):
    if n <= 1:
        return 0

    # Check if the graph is fully connected
    _, labels = connected_components(distance_matrix)
    if len(set(labels)) > 1:
        return 0

    # Calculate the number of unique paths
    return math.factorial(n - 1) // 2

# Function to calculate the total distance of a given path
def calculate_distance(path, distance_matrix):
    total_distance = 0
    for i in range(len(path) - 1):
        total_distance += distance_matrix[path[i]][path[i+1]]
    total_distance += distance_matrix[path[-1]][path[0]]
    return total_distance

# Function to solve TSP using dynamic programming (Held-Karp algorithm)
def tsp_dynamic_programming(distance_matrix):
    n = len(distance_matrix)
    # memoization table to store the minimum distance for each subset of nodes
    dp = [[float('inf')] * n for _ in range(1 << n)]
    # starting from the first city
    dp[1][0] = 0

    for mask in range(1 << n):
        for u in range(n):
            if mask & (1 << u):
                for v in range(n):
                    if mask & (1 << v) and u != v:
                        dp[mask][u] = min(dp[mask][u], dp[mask ^ (1 << u)][v] + distance_matrix[v][u])

    min_distance = min(dp[(1 << n) - 1][i] + distance_matrix[i][0] for i in range(1, n))
    path = []
    mask = (1 << n) - 1
    last = np.argmin([dp[mask][i] + distance_matrix[i][0] for i in range(n)])

    # Reconstruct the path
    while mask:
        path.append(last)
        next_mask = mask & ~(1 << last)
        for i in range(n):
            if mask & (1 << i) and dp[mask][last] == dp[next_mask][i] + distance_matrix[i][last]:
                last = i
                break
        mask = next_mask

    path.append(0)
    path = path[::-1]
    return [city + 1 for city in path], min_distance

n = 7

# Define the distance matrix for 7 cities (symmetric TSP)
distance_matrix = np.array([
    [0, 12, 10, float('inf'), float('inf'), float('inf'), 12],
    [12, 0, 8, 12, float('inf'), float('inf'), float('inf')],
    [10, 8, 0, 11, 3, float('inf'), 9],
    [float('inf'), 12, 11, 0, 11, 10, float('inf')],
    [float('inf'), float('inf'), 3, 11, 0, 6, 7],
    [float('inf'), float('inf'), float('inf'), 10, 6, 0, 9],
    [12, float('inf'), 9, float('inf'), 7, 9, 0]
])

# Replace infinity with a large value for MDS compatibility
large_value = np.max(distance_matrix[np.isfinite(distance_matrix)]) * 10
distance_matrix[np.isinf(distance_matrix)] = large_value

# Define city coordinates manually
cities = np.array([
    [0.1, 0.6],
    [0.4, 1],
    [0.38, 0.78],
    [0.78, 0.82],
    [0.5, 0.38],
    [0.8, 0],
    [0.24, 0.24]
])

# Normalize the city coordinates to [0, 1] range
cities = (cities - np.min(cities, axis=0)) / (np.max(cities, axis=0) - np.min(cities, axis=0))

# Parameters for the SOM
n_cities = cities.shape[0]
som = MiniSom(x=1, y=n_cities, input_len=2, sigma=1.0, learning_rate=0.5, topology='rectangular', random_seed=42)

# Initialize the weights
som.random_weights_init(cities)

# Train the SOM
som.train_random(cities, 1000)

# Get the winner nodes
route = []
for city in cities:
    winner = som.winner(city)
    route.append(winner[1])

# Sort the cities according to the route
route = np.argsort(route)
route = np.append(route, route[0])

# Calculate the SOM route distance
som_path = route + 1  # Convert to 1-based index for consistency
som_distance = calculate_distance(route, distance_matrix)

# Print the SOM path in ascending order
sorted_som_path = sorted(som_path)

# Plot the results using plotly
fig = go.Figure()

# Add city nodes
fig.add_trace(go.Scatter(x=cities[:, 0], y=cities[:, 1], mode='markers+text', text=[str(i+1) for i in range(n)],
                         textposition="top center", marker=dict(size=10, color='red'), name='Cities'))

# Add SOM route
fig.add_trace(go.Scatter(x=cities[route, 0], y=cities[route, 1], mode='lines+markers', line=dict(color='blue'), name='SOM Route'))

fig.update_layout(title='Traveling Salesman Problem - SOM Solution',
                  xaxis_title='Normalized X Coordinate',
                  yaxis_title='Normalized Y Coordinate',
                  showlegend=True)

fig.show()

# Calculate the number of unique paths considering connectivity
num_unique_paths = tsp_unique_paths(n, distance_matrix)

# Find the best path and minimum distance using dynamic programming (Held-Karp algorithm)
best_path, min_distance = tsp_dynamic_programming(distance_matrix)
print(f"Best path (dynamic programming): {best_path}")
print(f"Minimum distance (dynamic programming): {min_distance}")

# Print the SOM path and distance
print(f"SOM path: {som_path}")
print(f"SOM distance: {som_distance}")

# Print the sorted SOM path
print(f"Sorted SOM path: {sorted_som_path}")



Best path (dynamic programming): [1, 1, 3, 5, 7, 6, 4, 2]
Minimum distance (dynamic programming): 63.0
SOM path: [5 7 1 3 2 4 6 5]
SOM distance: 65.0
Sorted SOM path: [1, 2, 3, 4, 5, 5, 6, 7]
